## Graph Construction from Data:

In [1]:
# Loading data (~2 mins)
import pandas as pd

# Function to load a large CSV file
def load_large_csv(file_path):
    """
    Load a large CSV file.
    :param file_path: Path to the CSV file
    :return: DataFrame containing the dataset
    """
    ddf = pd.read_csv(file_path, low_memory=True)
    return ddf

# Load the large CSV file
file_path = 'Data/02-20-2018.csv'
df = load_large_csv(file_path)
display(df.head())
print(df.shape)
print(df.columns)


,Flow ID,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,172.31.69.25-94.231.103.172-22-45498-6,94.231.103.172,45498,172.31.69.25,22,6,20/02/2018 08:34:07,888751,11,11,...,32,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,Benign
1,8.0.6.4-8.6.0.1-0-0-0,8.6.0.1,0,8.0.6.4,0,0,20/02/2018 08:33:22,112642816,3,0,...,0,0.0,0.0,0.0,0.0,56300000.0,7.071068,56300000.0,56300000.0,Benign
2,8.0.6.4-8.6.0.1-0-0-0,8.6.0.1,0,8.0.6.4,0,0,20/02/2018 08:36:11,112642712,3,0,...,0,0.0,0.0,0.0,0.0,56300000.0,18.384776,56300000.0,56300000.0,Benign
3,8.0.6.4-8.6.0.1-0-0-0,8.6.0.1,0,8.0.6.4,0,0,20/02/2018 08:39:00,112642648,3,0,...,0,0.0,0.0,0.0,0.0,56300000.0,5.656854,56300000.0,56300000.0,Benign
4,8.0.6.4-8.6.0.1-0-0-0,8.6.0.1,0,8.0.6.4,0,0,20/02/2018 08:41:49,112642702,3,0,...,0,0.0,0.0,0.0,0.0,56300000.0,65.053824,56300000.0,56300000.0,Benign


(7948748, 84)
Index(['Flow ID', 'Src IP', 'Src Port', 'Dst IP', 'Dst Port', 'Protocol',
       'Timestamp', 'Flow Duration', 'Tot Fwd Pkts', 'Tot Bwd Pkts',
       'TotLen Fwd Pkts', 'TotLen Bwd Pkts', 'Fwd Pkt Len Max',
       'Fwd Pkt Len Min', 'Fwd Pkt Len Mean', 'Fwd Pkt Len Std',
       'Bwd Pkt Len Max', 'Bwd Pkt Len Min', 'Bwd Pkt Len Mean',
       'Bwd Pkt Len Std', 'Flow Byts/s', 'Flow Pkts/s', 'Flow IAT Mean',
       'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Tot',
       'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min',
       'Bwd IAT Tot', 'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max',
       'Bwd IAT Min', 'Fwd PSH Flags', 'Bwd PSH Flags', 'Fwd URG Flags',
       'Bwd URG Flags', 'Fwd Header Len', 'Bwd Header Len', 'Fwd Pkts/s',
       'Bwd Pkts/s', 'Pkt Len Min', 'Pkt Len Max', 'Pkt Len Mean',
       'Pkt Len Std', 'Pkt Len Var', 'FIN Flag Cnt', 'SYN Flag Cnt',
       'RST Flag Cnt', 'PSH Flag Cnt', 'ACK Flag Cnt', 'URG Flag Cnt',
       'CWE Flag

In [2]:
# Taking a subset of the data
df = df.iloc[:1000000, :]

In [3]:
df = df.dropna(subset=["Src IP", "Dst IP"])

print(df.shape)

# Find number of unique IPs in the dataset
unique_src_ips = df["Src IP"].nunique()
unique_dst_ips = df["Dst IP"].nunique()
print(f"Unique Source IPs: {unique_src_ips}")
print(f"Unique Destination IPs: {unique_dst_ips}")

(1000000, 84)
Unique Source IPs: 6800
Unique Destination IPs: 6753


In [4]:
# Saving the subset of the data
df.to_csv("Data/02-20-2018_subset.csv", index=False)

In [5]:
import networkx as nx

# Create directed graph
G = nx.DiGraph()

# Aggregate edge weights by total bytes
for _, row in df.iterrows():
    src = row["Src IP"]
    dst = row["Dst IP"]
    bytes_sent = row["TotLen Fwd Pkts"] + row["TotLen Bwd Pkts"]
    if G.has_edge(src, dst):
        G[src][dst]["weight"] += bytes_sent
    else:
        G.add_edge(src, dst, weight=bytes_sent)

# Print the number of nodes and edges in the graph
print(f"Number of nodes: {G.number_of_nodes()}")
print(f"Number of edges: {G.number_of_edges()}")

Number of nodes: 7558
Number of edges: 42673
